In [1]:
import os
import numpy as np
import pandas as pd

### Generate the bias logits

In [46]:
def read_biasfeatures(path):
    biasfeatures = pd.read_csv(path, index_col=0)
    return biasfeatures

In [48]:
qqp_train_biasfeatures = read_biasfeatures("/local/jyzhao/data/QQP/qqp_train_bias_features")

In [55]:
qqp_train_biasfeatures[:2]

,h-is-subseq,all-in-p,percent-in-p,log-len-diff,neg-in-h,label,average-sim,min-similarity,min2-similarity
0,0,0,0.363636,1.609438,0,0,0,0,0
1,0,0,0.571429,0.000000,0,1,0,0,0


In [61]:
bias_logits = []
for idx in range(len(qqp_train_biasfeatures)):
    instance = qqp_train_biasfeatures.iloc[idx]
    if instance["percent-in-p"] >= 0.7 and instance["label"] != 1:
        bias_logits.append([0.3, 0.7])
    else:
        bias_logits.append([0, 0])
with open("/local/jyzhao/data/QQP/bias_logits/pip_0.7_chanllenge", "w") as f:
    for line in bias_logits:
        f.write("\t".join([str(x) for x in line]) + "\n")

## Check the prediction acc

In [26]:
def read_pred(filepath):
    preds = []
    with open(filepath, "r") as f:
        lines = csv.reader(f, delimiter="\t", quotechar=None)
        for line in lines:
            pred_idx = np.argmax(np.array(line, dtype=float))
            preds.append(pred_idx)
    return preds
            

In [16]:
import csv

In [17]:
PAWS_QQP_PATH = "/local/jyzhao/data/paws/paws_qqp/output"

In [21]:
def get_test_examples(input_file, skipheader = False):
    with open(input_file, "r") as f:
        reader = csv.reader(f, delimiter="\t", quotechar=None)
        lines = []
        if skipheader:
            next(reader)
        for line in reader:
            lines.append(line)
    return lines

In [63]:
def get_acc(preds, test):
    assert len(preds) == len(test)
    preds = np.array(preds, dtype=float)
    test = np.array([x[-1] for x in test], dtype=float)
    correct = np.sum(preds == test)
    label_0 = np.where(test == 0)[0]
    label_1 = np.where(test ==1)[0]
    correct_0 = np.sum(preds[label_0] == test[label_0])
    correct_1 = np.sum(preds[label_1] == test[label_1])
    return (correct/len(preds), correct_0/len(label_0), correct_1/len(label_1))

In [64]:
qqp_paws_test = get_test_examples(os.path.join(PAWS_QQP_PATH, "dev_and_test.tsv"), True)

In [65]:
qqp_paws_test[:2]

[['1',
  "b'What were the major effects of the cambodia earthquake , and how do these effects compare to the Iquique earthquake in 1877 ?'",
  "b'What were the major effects of the Iquique earthquake , and how do these effects compare to the cambodia earthquake in 1877 ?'",
  '0'],
 ['2',
  'b"The guy I \'m dating never texts me and I feel like he does n\'t care about me but when I see him he shows me he likes me and wants me . Why do I feel this way ?"',
  'b"The guy I \'m dating never wants me and I feel like he does n\'t care about me but when I see him , he shows me he likes me and texts me . Why do I feel this way ?"',
  '0']]

In [66]:
qqp_paws_preds = read_pred("/local/jyzhao/bert/qqp_models/bert_base/test_results_paws_qqp_devtest.tsv")

In [67]:
qqp_paws_preds[:3]

[1, 1, 1]

In [68]:
acc_qqp_paws_test = get_acc(qqp_paws_preds, qqp_paws_test)
print(acc_qqp_paws_test)

(0.2998522895125554, 0.043209876543209874, 0.9528795811518325)


In [62]:
np.argmax([0.3, 0.7])

1

In [70]:
paws_wiki_test = get_test_examples("/local/jyzhao/data/Examples/PAWS_WIKI-examples/test.tsv", True)
paws_wiki_preds = read_pred("/local/jyzhao/bert/qqp_models/bert_base/test_results_paws_wiki_test.tsv")
get_acc(paws_wiki_preds, paws_wiki_test)

(0.474375, 0.15770609318996415, 0.8741515837104072)